In [81]:
import yaml
import copy
import os
import shutil
# Ruta del archivo YAML
ruta_fichero = 'federation.yaml'

# Leer el contenido del archivo YAML y almacenarlo en una variable
with open(ruta_fichero, 'r') as file:
    fdl= yaml.safe_load(file)

# fdl_federation tiene el contenido del archivo YAML
fdl_federation = yaml.dump(fdl, sort_keys=False)#print(fdl_federation)



In [82]:
orchestrator_name = list(fdl['functions']['oscar'][0].keys())[0]
print(orchestrator_name)
federation = fdl.get('functions', {}).get('oscar', [{}])[0].get(orchestrator_name, {}).get('federation', None)

if federation is not None:
    print("El atributo 'federation' existe.")
else:
    print("El atributo 'federation' no existe.")
    
ruta_carpeta = 'replicas'

# Verificar si la carpeta existe, si no, crearla
if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)
    print(f"La carpeta '{ruta_carpeta}' ha sido creada.")
else:
    print(f"La carpeta '{ruta_carpeta}' ya existe.")

oscar-replica1
El atributo 'federation' existe.
La carpeta 'replicas' ya existe.


In [83]:
orchestrator_name = list(fdl['functions']['oscar'][0].keys())[0]
print(orchestrator_name)

atrib_federation = fdl.get('functions', {}).get('oscar', [{}])[0].get(orchestrator_name, {}).get('federation', None)

if atrib_federation is not None:
    if fdl['functions']['oscar'][0][orchestrator_name]['federation'] and fdl['functions']['oscar'][0][orchestrator_name]['delegation'] != "static":
        print("ok")
    else:
        print('not')
else:
    print('not')

oscar-replica1
ok


In [84]:
# Cargar el YAML como un diccionario de Python

orchestrator= {
    "functions":{'oscar':list()}
}
# Obtener el primer cluster (orchestrator) y su nombre
save= list(fdl['functions']['oscar'])

# Creación de fdl_orchestrator por separado
orchestrator['functions']['oscar'].append((save[0]))

orchestrator['clusters']=copy.copy(fdl['clusters'])

orchestrator['storage_providers']=copy.copy(fdl['storage_providers'])

cluster_coordinator = yaml.dump(orchestrator, sort_keys=False)            
#print(cluster_coordinator)

directory = 'replicas'

# Verificar si la carpeta existe, si no, crearla
if not os.path.exists(directory):
    os.makedirs(directory)
    
ruta_archivo =directory+"/fedecluster_"+ orchestrator_name.replace("-", "_")+'_orchestrator.yaml'

with open(ruta_archivo, 'w') as file:
    yaml.dump(orchestrator, file, sort_keys=False)

print(f"FDL guardado en {ruta_archivo}")

if fdl['functions']['oscar'][0][orchestrator_name]['federation']:
    print(fdl['functions']['oscar'][0][orchestrator_name]['federation'])
else:
    print(fdl['functions']['oscar'][0][orchestrator_name]['replicas'])
    
    
orchestrator_replicas = fdl['functions']['oscar'][0][orchestrator_name]['replicas']
orchestrator_output = fdl['functions']['oscar'][0][orchestrator_name]['output']


# Crear un diccionario para mapear nombres de apartados a sus respectivos cluster_ids
section_names = [list(section.keys())[0] for section in fdl['functions']['oscar']]
name_to_section = {name: section for section in fdl['functions']['oscar'] for name in section.keys()}

delegation= fdl['functions']['oscar'][0][orchestrator_name]['delegation']
section_storage = list(fdl['storage_providers']['minio'])
#print(section_storage[0])

FDL guardado en replicas/fedecluster_oscar_replica1_orchestrator.yaml
True


In [85]:
for section in fdl['functions']['oscar'][1:]:
    element = {
         "functions":{'oscar':list()}
 }
    
    replica_name = list(section.keys())[0]
    #[section_name].append(['delegation']=copy.copy(delegation))
    section[replica_name]['federation']=copy.copy(fdl['functions']['oscar'][0][orchestrator_name]['federation'])
    section[replica_name]['delegation']=copy.copy(fdl['functions']['oscar'][0][orchestrator_name]['delegation'])
    section[replica_name]['replicas'] = copy.deepcopy(orchestrator_replicas)
    #section[section_name]['output'].append(first_output)
    # Verifica que 'output' sea una lista antes de intentar agregarle elementos
    if isinstance(section[replica_name]['output'], list):
        orchestrator_output_copy = copy.deepcopy(orchestrator_output)
        section[replica_name]['output'].extend(orchestrator_output_copy)
        for storage in section_storage:
                    if storage== section_names[0]:
                        minio_storage= "minio."+ storage
                        minio= section[replica_name]['output'][-1]
                        minio['storage_provider']= minio_storage
                        #first_output_copy['storage_provider'] = minio_storage
        
    else:
        raise TypeError(f"El valor de 'output' en {replica_name} no es una lista.")
    #print(section_name)
    # Modificar el 'cluster_id' en cada replica
    for  replica in section[replica_name]['replicas']:
        #section[section_name]['output']= first_output
        if replica['cluster_id'] in name_to_section:
            if replica['cluster_id'] == replica_name:
                replica['cluster_id'] = section_names[0]
                replica['service_name']= fdl['functions']['oscar'][0][orchestrator_name]['name']
                
   
    # Cargar datos para generar fdl de cada servicio que tiene el sistema de replicas
    element['functions']['oscar'].append( section)
    element['clusters']=copy.copy(fdl['clusters'])
    element['storage_providers']=copy.copy(fdl['storage_providers'])
    fdl_yaml_replica = yaml.dump(element, sort_keys=False)            
    #print(fdl_yaml_replica)
    ruta_archivo = directory+"/fedecluster_"+replica_name.replace("-", "_")+'.yaml'

    with open(ruta_archivo, 'w') as file:
        yaml.dump(element, file, sort_keys=False)

    print(f"FDL guardado en {ruta_archivo}")
                

# Convertir el diccionario modificado a YAML para ver el resultado
fdl_yaml_modified = yaml.dump(fdl, sort_keys=False)
#print(type(fdl_yaml_modified))

FDL guardado en replicas/fedecluster_oscar_local.yaml
FDL guardado en replicas/fedecluster_oscar_pi.yaml
FDL guardado en replicas/fedecluster_oscar_other.yaml


In [86]:
### Guardar el contenido en un archivo YAML
ruta_archivo = directory +'/created_federation.yaml'

with open(ruta_archivo, 'w') as file:
    yaml.dump(fdl, file, sort_keys=False)

print(f"FDL guardado en {ruta_archivo}")

FDL guardado en replicas/created_federation.yaml


In [91]:

add=0
# Listar los elementos del directorio
# Rutas de origen y destino
script=copy.copy(fdl['functions']['oscar'][0][orchestrator_name]['script'])
print(script)



# Copiar el archivo
try:
    shutil.copy(script, directory+'/'+script)
    print(f"Archivo {script} copiado a /{directory}.")
except FileNotFoundError:
    print(f"El archivo {script} no se encontró.")
except PermissionError:
    print(f"No se tienen permisos para copiar el archivo {script}.")
except Exception as e:
    print(f"Ocurrió un error: {e}")
if os.listdir(directory):
    for archive in os.listdir(directory):
        ruta_completa = os.path.join(directory, archive)
        fedecluster="fedecluster" in ruta_completa
        if os.path.isfile(ruta_completa) and ruta_completa.endswith('.yaml') and fedecluster :  # Verificar si es un archivo
            print(ruta_completa)
            add=add+1
if add==0:
    print("No existen ficheros yaml para ejecutar")

script.sh
Archivo script.sh copiado a /replicas.
replicas/fedecluster_oscar_pi.yaml
replicas/fedecluster_oscar_other.yaml
replicas/fedecluster_oscar_local.yaml
replicas/fedecluster_oscar_replica1_orchestrator.yaml


In [33]:
import subprocess

# Ejecutar un comando de consola (ejemplo: listar archivos en el directorio actual)
fdl_directory=directory+'/oscar_local.yaml'
resultado = subprocess.run(['oscar-cli', 'apply',directory], capture_output=True, text=True)

# Mostrar el resultado en consola
print(resultado.stdout)

Applying file "oscar_local.yaml"...

